<a href="https://colab.research.google.com/github/Vorlde/-WQU-MScFE-Capstone-Project/blob/main/Trading%20chkpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install dill
! pip install statsmodels
!pip install hurst
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller as ADF_test
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.models import load_model

# Define the file path for saving the model
file_path = "/content/drive/My Drive/Model/hedge_ratio_model.h5"

# To load the model later
loaded_model = load_model(file_path)


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing

from statsmodels.tsa.stattools import coint

from scipy import stats

import lzma
import dill as pickle

def save_pickle(path,obj):
    with lzma.open(path,"wb") as fp:
        pickle.dump(obj,fp)

def load_pickle(path):
    with lzma.open(path,"rb") as fp:
        file = pickle.load(fp)
    return file

def clean_data(ticker_dfs,tickers):
    intraday_range = ticker_dfs[tickers[0]].index
    for inst in tickers:
        ticker_dfs[inst] = ticker_dfs[inst].reindex(intraday_range)
    closes = []

    for tk in tickers:
        close = ticker_dfs[tk].close
        closes.append(close)

    pricing = pd.concat(closes,axis = 1)
    pricing.columns = tickers

    return pricing




def get_pca_features(ret_df,N_PRIN_COMPONENTS =10):

    pca = PCA(n_components=N_PRIN_COMPONENTS)
    pca.fit(ret_df)

    # Extract factor loadings
    factor_loadings = pca.components_.T  # Transpose the components matrix

    # Create a DataFrame with the correct orientation
    factor_loadings_df = pd.DataFrame(factor_loadings, index=ret_df.columns, columns=[f'Factor {i+1}' for i in range(N_PRIN_COMPONENTS)])

    X = preprocessing.StandardScaler().fit_transform(pca.components_.T)

    return X

def create_clusters(X,index):
    clf = DBSCAN(eps=1, min_samples=3)

    print(clf)

    clf.fit(X)
    labels = clf.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print("\nClusters discovered: %d" % n_clusters_)

    clustered = clf.labels_

    clustered_series = pd.Series(index=index, data=clustered.flatten())
    clustered_series = clustered_series[clustered_series != -1]

    return clustered_series


def find_cointegrated_pairs(data, significance=0.05):
    # This function is from https://www.quantopian.com/lectures/introduction-to-pairs-trading
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < significance:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs



def get_coint_pairs(prices,clustered_series):

    valid_tickers = clustered_series.index.intersection(prices.columns)
    clustered_series = clustered_series.loc[valid_tickers]

    CLUSTER_SIZE_LIMIT = 9999
    counts = clustered_series.value_counts()
    ticker_count_reduced = counts[(counts>1) & (counts<=CLUSTER_SIZE_LIMIT)]

    cluster_dict = {}
    for i, which_clust in enumerate(ticker_count_reduced.index):
        tickers = clustered_series[clustered_series == which_clust].index
        score_matrix, pvalue_matrix, pairs = find_cointegrated_pairs(
            prices[tickers]
        )
        cluster_dict[which_clust] = {}
        cluster_dict[which_clust]['score_matrix'] = score_matrix
        cluster_dict[which_clust]['pvalue_matrix'] = pvalue_matrix
        cluster_dict[which_clust]['pairs'] = pairs

    pairs = []
    for clust in cluster_dict.keys():
        pairs.extend(cluster_dict[clust]['pairs'])

    return pairs


def get_data():
    data_path = "/content/drive/My Drive/constituents.csv"
    dfs_path = "/content/drive/My Drive/new_dfs.obj"

    ticker_dfs = load_pickle(dfs_path)
    snp_data = pd.read_csv(data_path)
    tickers = []

    for i in range(499):
      tickers.append(snp_data.Symbol[i])

    tickers.remove("BF.B")
    tickers.remove("BRK.B")
    tickers.remove("CPAY")
    tickers.remove("DAY")
    tickers.remove("GEV")
    tickers.remove("SOLV")

    return tickers,ticker_dfs

In [5]:
tickers,ticker_dfs = get_data()

In [6]:
def generate_trading_data_blocks(df, month,year):
    if month == 12:
      month = 1
      year +=1

    else:
      month+=1

    month_data =df[(df.index.year == year) & (df.index.month == month)]
    coint_check_data = month_data[:500]
    trading = month_data

    return coint_check_data,trading


def calculate_monthly_clusters(monthly_data):
    """
    Calculate clusters for the given month.

    """
    ret_df = monthly_data.pct_change().round(4).fillna(0)
    X = get_pca_features(ret_df)
    clustered_series = create_clusters(X, ret_df.columns)

    return clustered_series


def yield_monthly_data(df):
    """
    Yields each month's data from the dataframe.
    """
    for year in df.index.year.unique():
        for month in df[df.index.year == year].index.month.unique():
            monthly_data = df[(df.index.year == year) & (df.index.month == month)]
            yield year, month, monthly_data

In [7]:

import statsmodels.api as sm
from hurst import compute_Hc  # Assuming you have a function to compute the Hurst exponent
from statsmodels.tools.tools import add_constant
from scipy.stats import skew, kurtosis

def add_stationary(spread_normalized):

    rolling_volatility = spread_normalized[:500].rolling(window=10).std()
    hurst_exponent = compute_Hc(spread_normalized[:500])[0]  # Hurst exponent

    skewness = skew(spread_normalized[:500])
    kurtosis_value = kurtosis(spread_normalized[:500])


    # Ensure spread_lagged starts from the second element to align with spread_diff which starts from the second element due to differencing and shifting
    spread_lagged = spread_normalized.shift(1).dropna()
    spread_diff = spread_normalized.diff().dropna()  # First difference to get spread_diff

    # Make sure indices are aligned by reindexing spread_diff to match spread_lagged
    spread_diff = spread_diff.reindex(spread_lagged.index)
    spread_diff += 1e-8

    # Now perform your regression with aligned indices
    model = sm.OLS(np.log(np.abs(spread_diff)), add_constant(spread_lagged))
    result = model.fit()
    phi = result.params[0]


    spread_normalized = spread_normalized[:500]  # Normalized spread

    features_df = pd.DataFrame({
        'spread_normalized': spread_normalized,
        'rolling_volatility': rolling_volatility[:500],
        'hurst_exponent': np.repeat(hurst_exponent, rolling_volatility.shape[0]),
    })

    # Mean Reversion Speed calculated as the absolute value of phi
    mean_reversion_speed = np.abs(phi)

    # Calculate half-life from the decay factor
    half_life = -np.log(2) / np.log(np.abs(phi))


    features_df['half_life'] = np.repeat(half_life, len(features_df))
    features_df['mean_reversion_speed'] = np.repeat(mean_reversion_speed, len(features_df))
    features_df['skewness'] = np.repeat(skewness, len(features_df))
    features_df['kurtosis'] = np.repeat(kurtosis_value, len(features_df))

    return features_df



# def process_pair(pair, prices):
#     s1, s2 = pair

#     datax = prices[s1]
#     datay = prices[s2]

#     # Find the hedge ratio using OLS on the first 500 data points
#     hedge_ratio = sm.OLS(datax[:500], sm.add_constant(datay[:500])).fit().params[1]

#     # Calculate the normalized spread based on the hedge ratio
#     spread = datax - hedge_ratio * datay
#     spread_normalized = (spread - spread.mean()) / spread.std()

#     features_df = add_stationary(spread_normalized)
#     features_df['hedge_ratio'] = np.repeat(hedge_ratio, len(features_df))

#     # Perform ADF test on the first 500 points for stationarity
#     adf_result_train = ADF_test(spread_normalized)
#     print(pair, adf_result_train[1], )

#     features_df.dropna(inplace = True)


#     # Proceed only if the first 500 points spread is stationary
#     if adf_result_train[1] < 0.05:
#         stationarity_train = adf_result_train[1] < 0.05

#         return features_df




def process_pair(pair, prices):
    s1, s2 = pair

    datax = prices[s1]
    datay = prices[s2]

    # Calculate hedge ratio using the first 500 data points
    hedge_ratio = sm.OLS(datax[:500], sm.add_constant(datay[:500])).fit().params[1]

    # Calculate the spread based on the hedge ratio
    spread = datax - hedge_ratio * datay

    # Calculate mean and standard deviation using the first 500 data points
    initial_mean = spread[:500].mean()
    initial_std = spread[:500].std()

    # Create a DataFrame with the calculated columns
    df = pd.DataFrame({
        'datax': datax,
        'datay': datay,
        'hedge_ratio': hedge_ratio,
        'spread': spread,
        'rolling_mean': initial_mean,
        'rolling_std': initial_std
    })

    # Normalized spread
    spread_normalized = (spread - initial_mean) / initial_std

    # Plotting the normalized spread along with mean and ±2 std lines
    plt.figure(figsize=(15, 7))
    spread_normalized = spread_normalized.reset_index(drop=True)
    plt.title(f'Z-Scores for {s1} and {s2} Across Training Period')
    spread_normalized.plot(label='Normalized Spread')
    plt.axhline(y=0, color='black', linestyle='--', label='Mean')
    plt.axhline(y=2, color='red', linestyle='--', label='+2 Std Dev')
    plt.axhline(y=-2, color='green', linestyle='--', label='-2 Std Dev')
    plt.legend()
    plt.show()

    return df


In [8]:
def main_driver_function(prices):

    for year, month, monthly_data in yield_monthly_data(prices):

        print(f"Clustering for {month} {year}")
        cluster = calculate_monthly_clusters(monthly_data)

        for block_data in generate_trading_data_blocks(prices, month, year):
            if block_data.shape[0] == 624:
                block_data = block_data.dropna(axis=1)
                pairs = get_coint_pairs(block_data, cluster)

                features_list, stationarity_labels, hedge_ratios = [], [], []
                for pair in pairs:
                    result = process_pair(pair, block_data)
                    if result is not None:
                        features_df, stationarity_test, hedge_ratio_test = result
                        features_list.append(features_df)
                        stationarity_labels.append(stationarity_test)
                        hedge_ratios.append(hedge_ratio_test)



In [9]:
import os

intraday_dfs = ticker_dfs.copy()
prices = clean_data(intraday_dfs, tickers)

# main_driver_function(prices[-100:])

In [12]:
import pandas as pd
import numpy as np

def calculate_spread(data, pair):
    y = data[pair[0]]
    x = data[pair[1]]
    return y - x

def calculate_rolling_stats(spread, lookback):
    rolling_mean = spread.rolling(window=lookback).mean()
    rolling_std = spread.rolling(window=lookback).std()
    return rolling_mean, rolling_std

def generate_signals(spread, rolling_mean, rolling_std, enter_threshold, exit_threshold):
    signals = pd.DataFrame(index=spread.index)
    signals['spread'] = spread
    signals['upper_limit'] = rolling_mean + enter_threshold * rolling_std
    signals['lower_limit'] = rolling_mean - enter_threshold * rolling_std
    signals['up_medium'] = rolling_mean + exit_threshold * rolling_std
    signals['low_medium'] = rolling_mean - exit_threshold * rolling_std

    signals['position'] = 0
    signals.loc[spread > signals['upper_limit'], 'position'] = -1
    signals.loc[spread < signals['lower_limit'], 'position'] = 1
    signals.loc[(spread < signals['up_medium']) & (spread > signals['low_medium']), 'position'] = 0

    signals['position'] = signals['position'].ffill().shift().fillna(0)

    return signals

def trade_pairs(pair, trading_data, lookback=500, enter_threshold=2, exit_threshold=0.5):
    spread = calculate_spread(trading_data, pair)
    rolling_mean, rolling_std = calculate_rolling_stats(spread, lookback)

    signals = generate_signals(spread, rolling_mean, rolling_std, enter_threshold, exit_threshold)

    trading_data['returns'] = trading_data[pair[0]].pct_change() - trading_data[pair[1]].pct_change()
    trading_data['strategy_returns'] = signals['position'] * trading_data['returns']

    trading_data['cumulative_returns'] = (1 + trading_data['strategy_returns']).cumprod() - 1
    total_profit = trading_data['cumulative_returns'].iloc[-1]

    return total_profit, trading_data


In [14]:
import warnings
warnings.filterwarnings("ignore")

data = prices[-5000:]

for year, month, monthly_data in yield_monthly_data(data):
    print(f"Clustering for {month} {year}")
    cluster = calculate_monthly_clusters(monthly_data)
    data = data.dropna(axis=1)
    coint_check_data, trading_data = generate_trading_data_blocks(data, month,year)
    pairs = get_coint_pairs(coint_check_data, cluster)

    print(pairs)

    #trade these pairs
    for pair in pairs:
      profits, results_df = trade_pairs(pair, trading_data)
      print(f'Total profit for the pair {pair}: {profits}')
      # print(results_df.tail())



Clustering for 9 2023
DBSCAN(eps=1, min_samples=3)

Clusters discovered: 5
[]
Clustering for 10 2023
DBSCAN(eps=1, min_samples=3)

Clusters discovered: 11
[]
Clustering for 11 2023
DBSCAN(eps=1, min_samples=3)

Clusters discovered: 7
[('MMM', 'PFE'), ('GOOGL', 'GOOG'), ('GOOG', 'PFE'), ('BAC', 'KO'), ('BAC', 'PFE'), ('BAC', 'VZ')]
Total profit for the pair ('MMM', 'PFE'): -0.02821281423895361
Total profit for the pair ('GOOGL', 'GOOG'): 0.004340305410650114
Total profit for the pair ('GOOG', 'PFE'): -0.006918976380576036
Total profit for the pair ('BAC', 'KO'): -0.013552159194277436
Total profit for the pair ('BAC', 'PFE'): -0.0775502061888721
Total profit for the pair ('BAC', 'VZ'): -0.05930318032167492
Clustering for 12 2023
DBSCAN(eps=1, min_samples=3)

Clusters discovered: 11


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
!pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 2.4 MB/s eta 0:00:00
